# Simple RAG Exercise

Build a simple RAG flow to recommend oldie movies based on user's requests. The dataset includes 5,000 movies descriptions. In the exercise, you will learn to add a filter to the semantic retrieval and the data columns sent to the generation step.

Fill in the empty cells, and answer the questions on the course site.

In [1]:
from rich.console import Console
from rich_theme_manager import Theme, ThemeManager
import pathlib

theme_dir = pathlib.Path("../themes")
theme_manager = ThemeManager(theme_dir=theme_dir)
dark = theme_manager.get("dark")

# Create a console with the dark theme
console = Console(theme=dark)

## Loading the Movie Dataset

We will load the moview dataset from Hugging Face hub in:
https://huggingface.co/datasets/AiresPucrs/tmdb-5000-movies

In [2]:
from datasets import load_dataset

### YOUR CODE HERE ###

In [ ]:
console.print(dataset)

## Encode using Vector Embedding

We will use one of the popular open source vector databases, [Qdrant](https://qdrant.tech/), and one of the popular embedding encoder and text transformer libraries, [SentenceTransformer](https://sbert.net/).

This time we will use the following sentence similarity model:
https://huggingface.co/sentence-transformers/all-mpnet-base-v2

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

# Create the embedding encoder
### YOUR CODE HERE ###


In [ ]:
console.print(encoder)

In [ ]:
# Create collection to store the wine rating data
collection_name="movies"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

### Loading the data into the vector database

We will use the collection that we created above, to go over all the rows and encode the `overview` column of the wine dataset, encode it with the encoder into embedding vector, and store it in the vector database. Please use the index of the movie from the dataset (`id` column) as the `id` in the vector index.

Please note that some of the rows are missing the `overview`. You should ignore them and not upload them into the vector database index.

This step will take a few seconds (less than a minute on my laptop).

In [7]:
# vectorize!
qdrant.upload_points(
    collection_name=collection_name,
    points=[
### YOUR CODE HERE ###
    ]
)

In [ ]:
console.print(
    qdrant
    .get_collection(
        collection_name=collection_name
    )
)

## **R**etrieve sematically relevant data based on user's query

Once the data is loaded into the vector database and the indexing process is done, we can start using our simple RAG system.

In [49]:
user_prompt = "Love story between an Asian king and European teacher"

### Encoding the user's query

We will use the same encoder that we used to encode the document data to encode the query of the user. 
This way we can search results based on semantic similarity. 

In [50]:
query_vector = encoder.encode(user_prompt).tolist()

### Create filter on the results

We only want movies from the '90s. Please create a filter base on the `release_date` column. Check the Qdrant documentation in: https://qdrant.tech/documentation/concepts/filtering/#datetime-range

In [51]:
from qdrant_client import models

query_filter= models.Filter(
### YOUR CODE HERE
    )

### Search similar rows

We can now take the embedding encoding of the user's query and use it to find similar rows in the vector database.

In [54]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1,
    query_filter=query_filter,
)

In [ ]:
from rich.text import Text
from rich.table import Table

table = Table(title="Retrieval Results", show_lines=True)

table.add_column("ID", style="#e0e0e0")
table.add_column("Original Title", style="#e0e0e0")
table.add_column("Overview", style="bright_red")
table.add_column("Score", style="#89ddff")

for hit in hits:
    table.add_row(
        str(hit.payload["id"]),
        hit.payload["original_title"],
        f'{hit.payload["overview"]}',
        f"{hit.score:.4f}"
    )

console.print(table)

## **A**ugment the prompt to the LLM with retrieved data

In our simple example, we will simply take the top result and use it in the prompt to the generation LLM. We will filter some of the columns and keep only the following:
* `original_title`
* `title`
* `overview`
* `release_date`
* `popularity`

In [56]:
# define a variable to hold the search results with specific fields
search_results = [
    {
### YOUR CODE HERE
    } for hit in hits]

In [ ]:
console.print(search_results)

## **G**enerate reply to the user's query

We will use GPT-4 from [OpenAI](https://platform.openai.com/docs/models). Please write the prompt to instruct the LLM to write the recommendations based on the search results.

In [58]:
from openai import OpenAI
from rich.panel import Panel

client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
### YOUR CODE HERE ###
    ]
)

response_text = Text(completion.choices[0].message.content)
styled_panel = Panel(
    response_text,
    title="Movie Recommendation with Retrieval",
    expand=False,
    border_style="bright_yellow",
    padding=(1, 1)
)

console.print(styled_panel)

╭────────────────────────────────────── Movie Recommendation with Retrieval ──────────────────────────────────────╮
│                                                                                                                 │
│ The movie you're looking for is 'Anna and the King' (1999). The film depicts the unique relationship between    │
│ the King of Siam (now Thailand) and a widowed British school teacher Anna Leonowens during the 1860's. Anna     │
│ teaches the king's many children and slowly develops a romantic yet controversial relationship with the king    │
│ himself. This beautifully crafted story takes you through a rollercoaster of emotions, intrigue, and discovery  │
│ of new cultures. You're sure to enjoy it.                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯